# Práctica 7:

En esta práctica se tiene como base los modelos lineales realizados en la práctica anterior, pero ahora se debe incluir un modelo de regresión múltiple para usar los resultados como clasificador a una variable de interés para este trabajo. 

La variable más importante en los datos utilizados en las prácticas anteriores, es el de diagnóstico, ya que uno de los objetivos pre vistos era el de encontrar información pertinente que nos ayude a hacer un clasificador de cáncer de mama, con las demás columnas que contiene el archivo csv con el que se ha estado trabajando.


In [1]:
import statsmodels.api as sm
from numpy import isnan
import pandas as pd
import ssl

In [4]:
if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
o = pd.read_csv("https://raw.githubusercontent.com/mayraberrones94/Ciencia_de_Datos/master/w-data.csv")

In [3]:
x = o.loc[o.diagnosis == 'M']

x = o.loc[o.diagnosis == 'B']
moi = pd.concat([x.loc[x.CF1op >= 70], x.loc[~isnan(x.CF2op)]]) # materiales con moi

KeyError: "None of [Float64Index([13.54, 13.08, 9.504, 13.03, 8.196, 12.05, 13.49, 11.76, 13.64,\n              11.94,\n              ...\n              9.333, 12.88, 10.29, 10.16, 9.423, 14.59, 11.51, 14.05,  11.2,\n               7.76],\n             dtype='float64', length=357)] are in the [index]"